<a href="https://colab.research.google.com/github/polevev/kaggle/blob/main/%D0%A0%D0%BE%D0%B6%D0%B4%D0%B0%D0%B5%D0%BC%D0%BE%D1%81%D1%82%D1%8C_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_excel('/content/новыеданные (4).xlsx')
df.sample(5)

In [ ]:
cols_to_fix = ['Численность', 'Среднедушевые доходы в месяц', 'Соотношение браков и разводов (кол-во разводов на 1000 браков)']
for col in cols_to_fix:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r'\s+', '', regex=True)
        .str.replace(',', '.', regex=False)
        .astype(float)
    )

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
import matplotlib.pyplot as plt

df.hist(bins = 50, figsize=(20,15))
plt.show()

In [ ]:
data = pd.read_excel('/content/данные_final.xlsx')
data

In [ ]:
data['Регион'].values

In [ ]:
regions = pd.read_csv('/content/region_mapping.csv')
regions

In [ ]:
import geopandas as gpd
gdf = gpd.read_file('/content/ru.geojson')
gdf['name'] = regions['Russian']

In [ ]:
df_stats = data
gdf['name'] = gdf['name'].str.strip().str.lower()
df_stats['Регион'] = df_stats['Регион'].str.strip().str.lower()

In [ ]:
gdf = gdf.merge(df_stats, left_on = 'name', right_on = 'Регион')

In [ ]:
gdf

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
gdf.plot(column='Общая площадь жилых помещений на жителя', ax=ax, edgecolor='black',
         cmap=plt.get_cmap('jet'), legend = True,
         legend_kwds={'label': "Общая площадь жилых помещений на жителя", 'shrink': 0.6})
         #vmin = 0,
         #vmax = 1)
plt.title("Общая площадь жилых помещений на жителя 2025")
plt.axis("off")
plt.show()

Уровень безработицы, темпы прироста и общие коэф рождаемости имеют много шума

In [ ]:
from sklearn.preprocessing import LabelEncoder

lencoder = LabelEncoder()

df['Регион'] = lencoder.fit_transform(df['Регион'])

In [ ]:
df['Регион'].value_counts()

In [ ]:
df.drop(columns=['Пользователи_онлайн_знакомств'], inplace = True)

In [ ]:
df

In [ ]:
df['Соотношение браков и разводов (кол-во разводов на 1000 браков)']=df['Соотношение браков и разводов (кол-во разводов на 1000 браков)'].apply(lambda x: x/1000)

In [ ]:
df

In [ ]:
df.corr(numeric_only=True)

In [ ]:
import seaborn as sns
sns.heatmap(df.corr(numeric_only=True))

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(df, figsize = (12 ,8))

In [ ]:
df

In [ ]:
X = df.drop(columns=['Соотношение браков и разводов (кол-во разводов на 1000 браков)', 'Общие коэф разводимости (число разводов на 1000 чел)', 'Общие коэф брачности (число браков на 1000 чел)'])

In [ ]:
y = df['Общие коэф разводимости (число разводов на 1000 чел)']

In [ ]:
from sklearn.preprocessing import StandardScaler

Scaler =  StandardScaler()


X_origin = Scaler.fit_transform(X)

In [ ]:
X_origin = pd.DataFrame(X_origin, columns = X.columns)

In [ ]:
X_origin

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_origin.values, y.values, test_size=0.4, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb

models = {
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'LinearRegression': LinearRegression(),
    'SVR': SVR(),
    'KNeighbors': KNeighborsRegressor(),
    'xgb': xgb.XGBRegressor(random_state=42)
}

In [ ]:
from sklearn.model_selection import cross_validate, TimeSeriesSplit
import numpy as np

results = {}
for name, model in models.items():
    # Используем TimeSeriesSplit и правильные ключи метрик
    tscv = TimeSeriesSplit(n_splits=5)
    scores = cross_validate(
        model,
        X_train,
        y_train,
        cv=tscv,
        scoring=['neg_mean_squared_error', 'r2']
    )
    results[name] = {
        'MSE': -np.mean(scores['test_neg_mean_squared_error']),
        'R²': np.mean(scores['test_r2'])
    }

results_df = pd.DataFrame.from_dict(results, orient='index')
results_df

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Определяем модель
rf_model = RandomForestRegressor(random_state=42)

# Определяем параметры для поиска
param_grid = {
    'n_estimators': [100, 200, 300, 500],  # Количество деревьев
    'max_depth': [10, 20, 30],  # Максимальная глубина деревьев
    'min_samples_split': [2, 5, 10],  # Минимальное количество сэмплов для разбиения узла
    'min_samples_leaf': [2, 4, 6],  # Минимальное количество сэмплов в листе
    'max_features': ['auto', 'sqrt', 'log2'],  # Количество признаков для поиска лучшего разбиения
}

# Инициализация GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=True)

# Обучение на данных
grid_search.fit(X_train, y_train)

# Вывод лучших параметров
print("Best parameters found: ", grid_search.best_params_)

# Оценка модели с лучшими гиперпараметрами
best_model = grid_search.best_estimator_
best_score = grid_search.best_score_

print(f"Best score (MSE): {best_score:.4f}")

In [ ]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
print(f'MAE:{mean_absolute_error(y_test, y_pred)}')
print(f'MSE:{mean_squared_error(y_test, y_pred)}')
print(f'R^2:{r2_score(y_test, y_pred)}')

In [ ]:
from sklearn.model_selection import GridSearchCV
model = xgb.XGBRegressor(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv = 5,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    verbose = 1
)

grid.fit(X_train, y_train)

best_model = grid.best_estimator_
print('best_params', grid.best_params_)

best_score = grid.best_score_

print(f"Best score (MSE): {best_score:.4f}")

In [ ]:
y_pred = best_model.predict(X_test)

print(f'MAE:{mean_absolute_error(y_test, y_pred)}')
print(f'MSE:{mean_squared_error(y_test, y_pred)}')
print(f'R^2:{r2_score(y_test, y_pred)}')

In [ ]:
xb_model = xgb.XGBRegressor(random_state=42)

xb_model.fit(X_train, y_train)

In [ ]:
y_pred = xb_model.predict(X_test)

print(f'MAE:{mean_absolute_error(y_test, y_pred)}')
print(f'MSE:{mean_squared_error(y_test, y_pred)}')
print(f'R^2:{r2_score(y_test, y_pred)}')

In [ ]:
test = pd.read_excel('/content/данные_last (2).xlsx')

In [ ]:
test

In [ ]:
X = test.drop(columns=['Общие коэф брачности (число браков на 1000 чел)', 'Общие коэф разводимости (число разводов на 1000 чел)','Соотношение браков и разводов (кол-во разводов на 1000 браков)'])

In [ ]:
X['Регион'] = lencoder.transform(X['Регион'])

In [ ]:
X

In [ ]:
X_scaled = Scaler.transform(X)

In [ ]:
X_scaled = pd.DataFrame(X_scaled, columns = X.columns)

In [ ]:
X_scaled

In [ ]:
y_final = best_model.predict(X_scaled)

In [ ]:
test['Общие коэф брачности (число браков на 1000 чел)'] = y_final

In [ ]:
y_final

In [ ]:
test['Общие коэф разводимости (число разводов на 1000 чел)'] = y_final

In [ ]:
test

In [ ]:
test.to_excel('данные_final.xlsx', index=False)

In [ ]:
import torch

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(num_classes, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32,1)
        )


    def forward(self, x):
      return (self.fc(x))

In [ ]:
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

In [ ]:
train_loader = torch.utils.data.DataLoader(list(zip(X_train_tensor, y_train_tensor)), batch_size=64, shuffle=True)

In [ ]:
X_train.shape

In [ ]:
model = Net(num_classes=10)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import tqdm
from tqdm.auto import tqdm

def train_stochastic(model, loader, criterion, optimizer, num_epoch):
    # цикл по количеству эпох обучения
    total_epoch_loss = []
    for t in tqdm(range(num_epoch)):
        epoch_loss = []

        # цикл по батчам даталоадера
        for X_batch, y_batch in loader:
            # Вычислим предсказания нашей модели
            y_batch = y_batch.unsqueeze(1)
            y_pred = model(X_batch)

            # Посчитаем значение функции потерь  на полученном предсказании
            loss = criterion(y_pred, y_batch)
            epoch_loss.append(loss.item())

            # Выполним подсчёт новых градиентов
            loss.backward()
            # Выполним шаг градиентного спуска
            optimizer.step()
            # Обнулим сохраненные у оптимизатора значения градиентов
            # перед следующим шагом обучения
            optimizer.zero_grad()

        print(f"{t} epoch: curr_loss = {np.mean(epoch_loss)}")
        total_epoch_loss.append(epoch_loss)

    plt.figure(figsize=(10,8))
    plt.plot(range(1, num_epoch+1), total_epoch_loss, 'b-', linewidth=2)
    plt.title('Training Loss Over Epochs', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.grid(True)
    plt.show()

    return model

In [ ]:
model = train_stochastic(model, train_loader, criterion, optimizer, 500)

In [ ]:
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [ ]:
with torch.no_grad():
    nn_prediction = model(torch.FloatTensor(X_test_tensor))
    nn_prediction = nn_prediction.tolist()

In [ ]:
print(f'MAE:{mean_absolute_error(y_test, nn_prediction)}')
print(f'MSE:{mean_squared_error(y_test, nn_prediction)}')
print(f'R^2:{r2_score(y_test, nn_prediction)}')